### In light of the scrutiny below, the data `CLEAN_JSON_FILEPATH` appears to be high fidelity as is.

In [1]:
import json
import pandas as pd
from load_scrape_data import get_clean_data_df  # TODO relative imports
from constants import (  # TODO relative imports
    CRIT_SCORE_K,
    CRIT_KS,
    DATE_K,
    DESCRIPTIVE_KS,
    DESC_K,
    DEV_K,
    ESRB_K,
    GENRE_K,
    NUM_CRIT_MIX_REVS_K,
    NUM_CRIT_REVS_KS,
    NUM_USER_MIX_REVS_K,
    NUM_USER_REVS_KS,
    PUB_K,
    TITLE_K,
    USER_SCORE_K,
    USER_KS,
)

In [2]:
df = get_clean_data_df()
df

,description,developer,publisher,date,title,critics_score,num_critic_mixed_reviews,num_critic_negative_reviews,num_critic_positive_reviews,num_user_mixed_reviews,...,on_ps5,on_ps_vita,on_ps1,on_psp,on_wii_u,on_wii,on_xbox_360,on_xbox_1,on_xbox_x,on_xbox
https://www.metacritic.com/game/burgertime/,"As the burger chef, you're out to build delici...",Data East,Mattel,NaT,BurgerTime,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
https://www.metacritic.com/game/project-electric-sheep/,None,Charisma Entertainment Limited,Charisma Entertainment,2023-11-29,Project Electric Sheep,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
https://www.metacritic.com/game/only-wish/,"Welcome to ""Only Wish"", the next-generation ad...",Victor's Game Venture,Victor's Game Venture,2023-11-29,Only Wish,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
https://www.metacritic.com/game/my-suika-watermelon-game/,None,Watashi Games,Watashi Games,2023-11-29,My Suika - Watermelon Game,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
https://www.metacritic.com/game/heartbeat-girlfriend/,None,Shenzhen Nakao Treasure Cooperation Network Co...,Shenzhen Nakao Treasure Cooperation Network Co...,2023-11-29,Heartbeat Girlfriend,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.metacritic.com/game/galactic-field/,None,Two Thousand and one entertainment studios,Two Thousand and one entertainment studios,2023-11-29,GALACTIC FIELD,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
https://www.metacritic.com/game/plutos-island/,None,Valentino Quijano-Moeckel,Valentino Quijano-Moeckel,2023-11-29,Pluto's Island,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
https://www.metacritic.com/game/escape-from-the-nursing-home/,None,Simul Games,Simul Games,2023-11-29,Escape from the Nursing Home,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False
https://www.metacritic.com/game/flat-spot/,None,Marciman Game Creations,Marciman Game Creations,2023-11-29,Flat Spot,None,None,None,None,None,...,False,False,False,False,False,False,False,False,False,False


### drop future (post dating the scrape) releases:

In [3]:
scrape_date = pd.to_datetime('2023-11-30')
dates = df[DATE_K]
future_date_mask = dates > scrape_date

drops = df.loc[future_date_mask, DATE_K]
df = df.loc[~future_date_mask]

print(f'dropped {len(drops)} data:')
pd.DataFrame({DATE_K: drops})

dropped 3 data:


,date
https://www.metacritic.com/game/cave-runner/,2023-12-14
https://www.metacritic.com/game/mechblaze/,2023-12-14
https://www.metacritic.com/game/outer-wilds-archaeologist-edition/,2023-12-07


### checking `None` occurrences:

In [4]:
na_counts = df.isna().sum()
non_zero_mask = na_counts != 0
na_counts = na_counts.loc[non_zero_mask]
pd.DataFrame({'na_counts': na_counts})

,na_counts
description,39752
developer,2968
publisher,2968
date,9402
critics_score,115734
num_critic_mixed_reviews,115732
num_critic_negative_reviews,115732
num_critic_positive_reviews,115732
num_user_mixed_reviews,116908
num_user_negative_reviews,116908


### first, logging in `metacritic_games_spider.py` precludes the need to scrutinize some of these correctly parsed fields ...

#### description:

In [5]:
na_desc_mask = pd.isna(df[DESC_K])
na_descs = df.loc[na_desc_mask, DESC_K]
print(f'some of the {len(na_descs)} null {DESC_K}s:')
pd.DataFrame({DESC_K: na_descs}).sample(10, random_state=1)

some of the 39752 null descriptions:


,description
https://www.metacritic.com/game/niidzuma-x-gibo/,None
https://www.metacritic.com/game/nobunaga-senki/,None
https://www.metacritic.com/game/otome-riron-to-sono-shuuhen-bon-voyage/,None
https://www.metacritic.com/game/shojo-gentei/,None
https://www.metacritic.com/game/war-on-paper/,None
https://www.metacritic.com/game/seirei-kishi-aquael-reizoku-no-hanayume/,None
https://www.metacritic.com/game/checkmate-ii/,None
https://www.metacritic.com/game/kung-fu-strike-the-warriors-rise-master-level/,None
https://www.metacritic.com/game/predator-simulator/,None
https://www.metacritic.com/game/idol-shimai-o-h-ni-produce/,None


#### date:

In [6]:
na_date_mask = pd.isna(df[DATE_K])
na_dates = df.loc[na_date_mask, DATE_K]
print(f'some of the {len(na_dates)} null {DATE_K}s:')
pd.DataFrame({DATE_K: na_dates}).sample(10, random_state=1)

some of the 9402 null dates:


,date
https://www.metacritic.com/game/official-us-playstation-magazine-44/,NaT
https://www.metacritic.com/game/2-disney-games-phineas-and-ferb-plus-phineas-and/,NaT
https://www.metacritic.com/game/the-maze-1996/,NaT
https://www.metacritic.com/game/berlin-1948/,NaT
https://www.metacritic.com/game/mah-jong-medley/,NaT
https://www.metacritic.com/game/tommys-gin-rummy/,NaT
https://www.metacritic.com/game/official-uk-playstation-magazine-56/,NaT
https://www.metacritic.com/game/deep-sea-trophy-fishing/,NaT
https://www.metacritic.com/game/a-ressha-de-ikou-4/,NaT
https://www.metacritic.com/game/gabriel-knight-mysteries/,NaT


#### esrb:

In [7]:
na_esrb_mask = pd.isna(df[ESRB_K])
na_esrbs = df.loc[na_esrb_mask, ESRB_K]
print(f'some of the {len(na_esrbs)} null {ESRB_K}s:')
pd.DataFrame({ESRB_K: na_esrbs}).sample(10, random_state=1)

some of the 104841 null esrbs:


,esrb
https://www.metacritic.com/game/extreme-offroad-simulator/,None
https://www.metacritic.com/game/clashbowl/,None
https://www.metacritic.com/game/the-night-christmas-ended/,None
https://www.metacritic.com/game/tic-tac-show/,None
https://www.metacritic.com/game/mothlight/,None
https://www.metacritic.com/game/witches-x-warlocks/,None
https://www.metacritic.com/game/shadows-on-the-vatican-nightingale/,None
https://www.metacritic.com/game/tutankhamen-no-nazo-ankh/,None
https://www.metacritic.com/game/shin-koihime-plus-musou-premium-package-go/,None
https://www.metacritic.com/game/td-worlds/,None


### the `None` occurrences to scrutinize:

In [8]:
ignored_fields = [DATE_K, DESC_K, ESRB_K]
na_counts.drop(ignored_fields, inplace=True)
pd.DataFrame({'na_counts': na_counts})

,na_counts
developer,2968
publisher,2968
critics_score,115734
num_critic_mixed_reviews,115732
num_critic_negative_reviews,115732
num_critic_positive_reviews,115732
num_user_mixed_reviews,116908
num_user_negative_reviews,116908
num_user_positive_reviews,116908
users_score,116908


#### `None` publishers & developers always co-occur & cursory inspection shows they're correctly parsed:

In [9]:
num_na_devs = na_counts[DEV_K]
num_na_pubs = na_counts[PUB_K]

dev_and_pub_ks = [DEV_K, PUB_K]
na_masks = [
    pd.isna(df[k])
    for k in dev_and_pub_ks
]
na_dev_and_pub_mask = pd.concat(na_masks, axis=1).all(axis=1)
na_dev_and_pub_df = df.loc[na_dev_and_pub_mask]
num_na_devs_and_pubs = len(na_dev_and_pub_df)

if num_na_devs != num_na_pubs or num_na_devs_and_pubs != num_na_devs:
    raise ValueError('the above assertion is False')

na_dev_and_pub_df = df.loc[na_dev_and_pub_mask]
print(f'some of the {len(na_dev_and_pub_df)} data w/ all None {dev_and_pub_ks}:')
na_dev_and_pub_df[[DEV_K, PUB_K]].sample(10, random_state=1)

some of the 2968 data w/ all None ['developer', 'publisher']:


,developer,publisher
https://www.metacritic.com/game/dormitory/,None,None
https://www.metacritic.com/game/hakoniwa-tetsudou-blue-train-tokkyuuhen/,None,None
https://www.metacritic.com/game/bomb-boat/,None,None
https://www.metacritic.com/game/ryouki-no-ori-dai-4-shou/,None,None
https://www.metacritic.com/game/word-mojo/,None,None
https://www.metacritic.com/game/80-days/,None,None
https://www.metacritic.com/game/pro-yakyuu-team-de-asobou/,None,None
https://www.metacritic.com/game/hidden-object-classic-mysteries-4/,None,None
https://www.metacritic.com/game/igo-2006/,None,None
https://www.metacritic.com/game/slot-pro-dx-fujiko-2/,None,None


#### data w/ `None` critics_score is a superset of all cases where all `NUM_CRIT_REVS_KS` are `None`:

In [10]:
na_crit_score_mask = pd.isna(df[CRIT_SCORE_K])
na_crit_score_df = df.loc[na_crit_score_mask]

na_num_crit_revs_masks = [
    pd.isna(na_crit_score_df[num_crit_rev_k])
    for num_crit_rev_k in NUM_CRIT_REVS_KS
]
all_na_num_crit_revs_mask = pd.concat(na_num_crit_revs_masks, axis=1).all(axis=1)
all_na_num_crit_revs_df = na_crit_score_df.loc[all_na_num_crit_revs_mask]

na_num_crit_revs_counts = {na_counts[num_crit_rev_k] for num_crit_rev_k in NUM_CRIT_REVS_KS}
if len(na_num_crit_revs_counts) != 1:
    raise ValueError('the above assertion is False')

is_superset = len(all_na_num_crit_revs_df) == na_num_crit_revs_counts.pop()
if not is_superset:
    raise ValueError('the above assertion is False')

#### the minority cases of `None` critics_score w/ not all `None` `NUM_CRIT_REVS_KS` reveal the requirement for ≥ 4 reviews to assign a critics_score:

In [11]:
minority_cases_df = na_crit_score_df.loc[~all_na_num_crit_revs_mask]
print(f'the {len(minority_cases_df)} data w/ None {CRIT_SCORE_K} & not all None {NUM_CRIT_REVS_KS}:')
minority_cases_df[list(CRIT_KS)]

the 2 data w/ None critics_score & not all None {'num_critic_positive_reviews', 'num_critic_mixed_reviews', 'num_critic_negative_reviews'}:


,critics_score,num_critic_positive_reviews,num_critic_negative_reviews,num_critic_mixed_reviews
https://www.metacritic.com/game/total-war-shogun-2-rise-of-the-samurai/,None,2,0,1
https://www.metacritic.com/game/memento-mori-2/,None,1,0,2


#### cursory inspection of all other cases (all `NUM_CRIT_REVS_KS` are `None`) shows they're correctly parsed:

In [12]:
majority_cases_df = na_crit_score_df.loc[all_na_num_crit_revs_mask]
print(f'some of the {len(majority_cases_df)} data w/ all None {NUM_CRIT_REVS_KS}:')
majority_cases_df[list(USER_KS)].sample(10, random_state=1)

some of the 115732 data w/ all None {'num_critic_positive_reviews', 'num_critic_mixed_reviews', 'num_critic_negative_reviews'}:


,num_user_mixed_reviews,num_user_negative_reviews,users_score,num_user_positive_reviews
https://www.metacritic.com/game/doko-demo-issho-lets-gakkou/,None,None,None,None
https://www.metacritic.com/game/airborne-sea/,None,None,None,None
https://www.metacritic.com/game/iki-sugi-ibunka-kouryuu-seiso-hitozuma-ntr-ochii/,None,None,None,None
https://www.metacritic.com/game/solbrain-ix-forest/,None,None,None,None
https://www.metacritic.com/game/oshare-hamster-to-kurasou-issho-ni-odekake/,None,None,None,None
https://www.metacritic.com/game/ao-no-6-gou-antarctica/,None,None,None,None
https://www.metacritic.com/game/downtown-nekketsu-koushinkyoku-soreyuke/,None,None,None,None
https://www.metacritic.com/game/kinoman-vs-gray-elephant/,None,None,None,None
https://www.metacritic.com/game/vr-mini-world-roller-coaster/,None,None,None,None
https://www.metacritic.com/game/invocation-the-festival-of-souls/,None,None,None,None


#### data where all `NUM_USER_REVS_KS` are `None` is the set of all `None` users_score & cursory inspection shows they're correctly parsed:

In [13]:
na_user_ks_masks = [
    pd.isna(df[user_k])
    for user_k in list(USER_KS)
]
na_all_user_ks_mask = pd.concat(na_user_ks_masks, axis=1).all(axis=1)
na_all_user_ks_df = df.loc[na_all_user_ks_mask]


is_superset = len(na_all_user_ks_df) == na_counts[USER_SCORE_K]
if not is_superset:
    raise ValueError('the above assertion is False')

print(f'some of the {len(na_all_user_ks_df)} data w/ all None {USER_KS}:')
na_all_user_ks_df[list(USER_KS)].sample(10, random_state=1)

some of the 116908 data w/ all None {'num_user_mixed_reviews', 'num_user_negative_reviews', 'users_score', 'num_user_positive_reviews'}:


,num_user_mixed_reviews,num_user_negative_reviews,users_score,num_user_positive_reviews
https://www.metacritic.com/game/jk-to-ero-ryman-2/,None,None,None,None
https://www.metacritic.com/game/ensom-warrior/,None,None,None,None
https://www.metacritic.com/game/love-hentai-fap-fast-die-young/,None,None,None,None
https://www.metacritic.com/game/funny-panda/,None,None,None,None
https://www.metacritic.com/game/demon-bane/,None,None,None,None
https://www.metacritic.com/game/super-army-of-tentacles-not-a-cthulhu-dating-sim/,None,None,None,None
https://www.metacritic.com/game/beat-beauty/,None,None,None,None
https://www.metacritic.com/game/the-spike/,None,None,None,None
https://www.metacritic.com/game/shores-unknown/,None,None,None,None
https://www.metacritic.com/game/darts-up/,None,None,None,None


### checking duplicates among qualitiative fields:

#### duplicate descriptions are valid, but signs of low effort...

In [14]:
desc_counts = df[DESC_K].value_counts()
multi_descs_counts_mask = desc_counts > 1
multi_descs_counts = desc_counts.loc[multi_descs_counts_mask]
pd.DataFrame({'count': multi_descs_counts}).head(10)

,count
description,
This game is a classic puzzle game with several levels of difficulty and beautiful arts.,29
This game is a classic memory game with several levels of difficulty and beautiful arts.,28
Find the objects that are hidden on the map.,19
Minimal physical puzzle with explosions.,13
What could be more beautiful than hentai?,10
Help the beautiful girls to immerse themselves in their memories and remember the wonderful moments of their lives. This game is a classic puzzle game with several levels of difficulty and beautiful arts.,10
The game is a classic puzzle with three levels of difficulty.,9
Enjoy the beauty girls picture and relaxing music to complete the puzzle.,8
Help the beautiful girls to immerse themselves in their memories and remember the wonderful moments of their lives.\n\nThis game is a classic memory game with several levels of difficulty and beautiful arts.,8


#### ...they tend to be recycled tile sliding puzzles depicting chesty anime girls:

In [15]:
desc = 'The game is a classic puzzle with three levels of difficulty.'
has_desc_mask = df[DESC_K] == desc
df.loc[has_desc_mask, DESCRIPTIVE_KS + list(NUM_CRIT_REVS_KS) + list(NUM_USER_REVS_KS)]

,date,description,developer,esrb,genre,publisher,title,num_critic_positive_reviews,num_critic_mixed_reviews,num_critic_negative_reviews,num_user_negative_reviews,num_user_mixed_reviews,num_user_positive_reviews
https://www.metacritic.com/game/hentai-cosplay-elf/,2019-09-27,The game is a classic puzzle with three levels...,IR Studio,None,Puzzle,IR Studio,Hentai Cosplay Elf,None,None,None,None,None,None
https://www.metacritic.com/game/nyasha-winter/,2019-03-14,The game is a classic puzzle with three levels...,Zloy Krot Studio,None,Puzzle,IR Studio,Nyasha Winter,None,None,None,None,None,None
https://www.metacritic.com/game/nyasha/,2019-03-06,The game is a classic puzzle with three levels...,Zloy Krot Studio,None,Puzzle,IR Studio,Nyasha,None,None,None,None,None,None
https://www.metacritic.com/game/nya-nya-nya-girls/,2018-12-08,The game is a classic puzzle with three levels...,Zloy Krot Studio,None,Visual Novel,Zloy Krot Studio,Nya Nya Nya Girls,None,None,None,None,None,None
https://www.metacritic.com/game/nyasha-beach/,2019-03-27,The game is a classic puzzle with three levels...,Zloy Krot Studio,None,Puzzle,IR Studio,Nyasha Beach,None,None,None,None,None,None
https://www.metacritic.com/game/nyasha-land-of-elves/,2019-04-13,The game is a classic puzzle with three levels...,Zloy Krot Studio,None,Puzzle,IR Studio,Nyasha Land of Elves,None,None,None,None,None,None
https://www.metacritic.com/game/nyasha-valkyrie/,2019-04-19,The game is a classic puzzle with three levels...,Zloy Krot Studio,None,Puzzle,IR Studio,Nyasha Valkyrie,None,None,None,None,None,None
https://www.metacritic.com/game/elemental-girls/,2019-05-15,The game is a classic puzzle with three levels...,Zloy Krot Studio,None,Logic Puzzle,IR Studio,Elemental Girls,None,None,None,None,None,None
https://www.metacritic.com/game/hentai-halloween/,2019-10-28,The game is a classic puzzle with three levels...,IR Studio,None,Visual Novel,IR Studio,Hentai Halloween,None,None,None,None,None,None


#### surprisingly, all titles are unique:

In [16]:
title_counts = df[TITLE_K].value_counts()
title_multi_counts_mask = title_counts > 1
title_multi_counts = title_counts.loc[title_multi_counts_mask]
multi_count_titles = set(title_multi_counts.values)
print(f'there are {len(multi_count_titles)} multi count titles')

there are 1 multi count titles


#### the most prolific developer is unfamiliar to me & in its own league...

In [17]:
dev_counts = df[DEV_K].value_counts()
dev_multi_counts_mask = dev_counts > 1
dev_multi_counts = dev_counts.loc[dev_multi_counts_mask]
multi_count_descs = set(dev_multi_counts.values)
pd.DataFrame({'count': dev_multi_counts}).head(10)

,count
developer,
Big Fish Games,1072
Various,955
Konami,499
Capcom,403
Bandai Namco Games,374
Nintendo,343
Sega,305
Hamster,293
Ubisoft,260


#### ...also due to recycling low effort games of the "puzzle" & "hidden object" genres:

In [18]:
dev = 'Big Fish Games'
from_dev_mask = df[DEV_K] == dev
df.loc[from_dev_mask, DESCRIPTIVE_KS].sample(10, random_state=1)

,date,description,developer,esrb,genre,publisher,title
https://www.metacritic.com/game/the-curio-society-eclipse-over-mesina/,2016-06-05,After you're contacted by a mysterious secret ...,Big Fish Games,None,Hidden Object,Big Fish Games,The Curio Society: Eclipse Over Mesina
https://www.metacritic.com/game/hexcite/,2007-05-15,None,Big Fish Games,None,Puzzle,Big Fish Games,Hexcite
https://www.metacritic.com/game/witchs-tales/,2017-10-31,None,Big Fish Games,None,Puzzle,Big Fish Games,Witch's Tales
https://www.metacritic.com/game/shadowplay-harrowstead-mystery/,2019-06-15,You and your friends have been researching the...,Big Fish Games,None,Hidden Object,Big Fish Games,Shadowplay: Harrowstead Mystery
https://www.metacritic.com/game/secret-diaries-florence-ashford/,2010-10-16,None,Big Fish Games,None,Puzzle,Big Fish Games,Secret Diaries - Florence Ashford
https://www.metacritic.com/game/mahjongg-dimensions-deluxe-tiles-in-time/,2011-10-07,None,Big Fish Games,None,Matching Puzzle,Big Fish Games,Mahjongg Dimensions Deluxe: Tiles in Time
https://www.metacritic.com/game/witchcraft-the-lotus-elixir/,2014-07-29,An uncanny adventure is near! Dear Witch Appre...,Big Fish Games,None,Puzzle,Big Fish Games,Witchcraft: The Lotus Elixir
https://www.metacritic.com/game/search-for-the-wonderland/,2015-12-08,None,Big Fish Games,None,Puzzle,Big Fish Games,Search for the Wonderland
https://www.metacritic.com/game/wilderness-mosaic-4-easter-island/,2021-03-30,"After some exciting adventures, Claire is read...",Big Fish Games,None,Puzzle,Big Fish Games,Wilderness Mosaic 4: Easter Island
https://www.metacritic.com/game/the-secret-order-5-the-buried-kingdom/,2016-11-13,"After the Griffin Order banned time travel, Sa...",Big Fish Games,None,Hidden Object,Big Fish Games,The Secret Order 5: The Buried Kingdom


#### "Various" is also a valid value, largely by merit of being a collection (compilation or demo):

In [19]:
dev = 'Various'
from_dev_mask = df[DEV_K] == dev
various_dev_df = df.loc[from_dev_mask]
pd.DataFrame({'counts': various_dev_df['genre'].value_counts()})

,counts
genre,
Compilation,688
Demo Disc,258
FPS,4
Tycoon,2
Gambling,1
Aircraft Combat Sim,1
Management,1


#### publisher is a retread of developers: "Big Fish Games", followed by the reputable giants:

In [20]:
pub_counts = df[PUB_K].value_counts()
pub_multi_counts_mask = pub_counts > 1
pub_multi_counts = pub_counts.loc[pub_multi_counts_mask]
multi_count_descs = set(pub_multi_counts.values)
pd.DataFrame({'count': pub_multi_counts}).head(10)

,count
publisher,
Big Fish Games,3725
Nintendo,1065
Microsoft Game Studios,1059
Ubisoft,1035
Konami,1020
Sega,974
Electronic Arts,880
Bandai Namco Games,834
SCEA,716
